In [9]:
import cv2
import os
import imutils

#emotionName = 'Enojo'
#emotionName = 'Felicidad'
#emotionName = 'Sorpresa'
emotionName = 'Tristeza'

dataPath = r'D:\tec\9no Semestre\IA\Proyectos finales\caras\smg' 
emotionsPath = os.path.join(dataPath, emotionName)
if not os.path.exists(emotionsPath):
    print('Carpeta creada: ', emotionsPath)
    os.makedirs(emotionsPath)
    
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
count = 0

while True:
    ret, frame = cap.read()
    if not ret: 
        break
    frame = imutils.resize(frame, width=640)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    auxFrame = frame.copy()
    faces = faceClassif.detectMultiScale(gray, 1.3, 5)
    
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        rostro = auxFrame[y:y + h, x:x + w]
        rostro = cv2.resize(rostro, (150, 150), interpolation=cv2.INTER_CUBIC)
        cv2.imwrite(os.path.join(emotionsPath, 'rostro_{}.jpg'.format(count)), rostro)
        count += 1
    
    cv2.imshow('frame', frame)
    k = cv2.waitKey(1)
    if k == 27 or count >= 350:
        break

cap.release()
cv2.destroyAllWindows()

Carpeta creada:  D:\tec\9no Semestre\IA\Proyectos finales\caras\smg\Tristeza


In [13]:
import cv2
import os
import numpy as np
import time

def obtenerModelo(method, facesData, labels): 
    if method == 'LBPH':
        emotion_recognizer = cv2.face.LBPHFaceRecognizer_create()

    print("Entrenando (" + method + ")...")
    inicio = time.time()
    emotion_recognizer.train(facesData, np.array(labels))
    tiempoEntrenamiento = time.time() - inicio
    print("Tiempo de entrenamiento (" + method + "): ", tiempoEntrenamiento)

    ruta_modelo = r"D:\tec\9no Semestre\IA\Proyectos finales\caras\smg\model" + method + ".xml"
    emotion_recognizer.write(ruta_modelo)

dataPath = r'D:\tec\9no Semestre\IA\Proyectos finales\caras\smg' 
emotionsList = os.listdir(dataPath)
print('Lista de emociones: ', emotionsList)

labels = []
facesData = []
label = 0

for nameDir in emotionsList:
    emotionsPath = os.path.join(dataPath, nameDir)
    for fileName in os.listdir(emotionsPath):
        labels.append(label)
        facesData.append(cv2.imread(os.path.join(emotionsPath, fileName), 0))
    label += 1

obtenerModelo('LBPH', facesData, labels)


Lista de emociones:  ['Enojo', 'Felicidad', 'Sorpresa', 'Tristeza']
Entrenando (LBPH)...
Tiempo de entrenamiento (LBPH):  1.600480318069458


In [24]:
import cv2
import os
import numpy as np

method = 'LBPH'

if method == 'LBPH': 
    emotion_recognizer = cv2.face.LBPHFaceRecognizer_create()

model_path = r'D:\tec\9no Semestre\IA\Proyectos finales\caras\model' + method + '.xml'

if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model file not found: {model_path}")

emotion_recognizer.read(model_path)

dataPath = r'D:\tec\9no Semestre\IA\Proyectos finales\caras\smg'  
imagePaths = os.listdir(dataPath)
print('imagePaths =', imagePaths)

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

while True:
    ret, frame = cap.read()
    if not ret: 
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    auxFrame = gray.copy()
    faces = faceClassif.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        rostro = auxFrame[y:y+h, x:x+w]
        rostro = cv2.resize(rostro, (150, 150), interpolation=cv2.INTER_CUBIC)
        result = emotion_recognizer.predict(rostro)
        cv2.putText(frame, '{}'.format(result), (x, y-5), 1, 1.3, (255, 255, 0), 1, cv2.LINE_AA)
        
        if result[1] < 80:  
            cv2.putText(frame, '{}'.format(imagePaths[result[0]]), (x, y-25), 2, 1.1, (0, 255, 0), 1, cv2.LINE_AA)
        else:
            cv2.putText(frame, 'Desconocido', (x, y-25), 2, 1.1, (0, 0, 255), 1, cv2.LINE_AA)

        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    cv2.imshow('frame', frame)
    k = cv2.waitKey(1)
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()


imagePaths = ['Enojo', 'Felicidad', 'Sorpresa', 'Tristeza']
